<a href="https://colab.research.google.com/github/Spandana2704/ML/blob/main/ML(W4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/content/titanic_train.csv')
df.shape

(891, 12)

In [ ]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
df=df.drop(['PassengerId','Name','Ticket'],axis=1)

In [ ]:
df.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [ ]:
[col for col in df.columns if df[col].isnull().sum()>0]

['Age', 'Cabin', 'Embarked']

In [ ]:
df['Age'] = df['Age'].fillna(df['Age'].median())

In [ ]:
df=df.drop(['Embarked','Cabin'],axis=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [ ]:
X = df.drop('Survived',axis=1)
y = df['Survived']

In [ ]:
X = pd.get_dummies(X,drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
grouped_feature_ls = []
correlated_groups = []
def correlation (dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname=corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr
corr_features=correlation (x_test, 0.9)
correlated_features = set(corr_features)
print(len(correlated_features))

0


In [ ]:
sfs1=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=True,verbose=2,scoring='roc_auc' or 'accuracy',cv=3)

In [ ]:
sfs1=sfs1.fit(np.array(x_train.fillna(0)),y_train)

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    3.9s finished

[2025-08-19 15:17:35] Features: 1/5 -- score: 0.765194136612553[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.3s finished

[2025-08-19 15:17:39] Features: 2/5 -- score: 0.832223720877483[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.6s finished

[2025-08-19 15:17:41] Features: 3/5 -- score: 0.8391936829655554[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.5s finished

[2025-08-19 15:17:44] Features: 4/5 -- score: 0.8452480476977415[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.4s finished

[2025-08-19 15:17:45] Features: 5/5 -- score: 0.8450046174783709

In [ ]:
print('best accracy score',sfs1.k_score_)
print('best subset(indices)',sfs1.k_feature_idx_)
print('Best subset (corresponding names):', sfs1.k_feature_names_)

best accracy score 0.8450046174783709
best subset(indices) (0, 1, 3, 4, 5)
Best subset (corresponding names): ('0', '1', '3', '4', '5')


In [ ]:
sfs2=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=False,verbose=2,scoring='roc_auc' or 'accuracy',cv=3)
sfs2=sfs2.fit(np.array(x_train.fillna(0)),y_train)
print('best accracy score',sfs2.k_score_)
print('best subset(indices)',sfs2.k_feature_idx_)
print('Best subset (corresponding names):', sfs2.k_feature_names_)

best accracy score 0.8440774764265578
best subset(indices) (0, 1, 2, 4, 5)
Best subset (corresponding names): ('0', '1', '2', '4', '5')


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    4.1s finished

[2025-08-19 15:18:28] Features: 5/5 -- score: 0.8440774764265578

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
efs=EFS(RandomForestClassifier(n_jobs=4),min_features=1,max_features=4,scoring='accuracy',cv=5)
efs=efs.fit(x_train,y_train)
print('best accuracy score',efs.best_score_)
print('best subset(indices)',efs.best_idx_)
print('Best subset (corresponding names):', efs.best_feature_names_)

Features: 56/56

best accuracy score 0.8089806451612903
best subset(indices) (0, 4, 5)
Best subset (corresponding names): ('Pclass', 'Fare', 'Sex_male')


In [ ]:
efs_results = pd.DataFrame.from_dict(efs.get_metric_dict()).T
efs_results.sort_values(by='avg_score', ascending=True, inplace=True)
efs_results

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(1,)","[0.592, 0.576, 0.608, 0.5645161290322581, 0.60...",0.589071,"(Age,)",0.021425,0.016669,0.008335
11,"(1, 2)","[0.64, 0.568, 0.608, 0.5483870967741935, 0.612...",0.595458,"(Age, SibSp)",0.042299,0.03291,0.016455
12,"(1, 3)","[0.56, 0.592, 0.624, 0.6209677419354839, 0.612...",0.601974,"(Age, Parch)",0.03056,0.023777,0.011888
34,"(1, 3, 4)","[0.584, 0.6, 0.624, 0.5967741935483871, 0.6209...",0.605148,"(Age, Parch, Fare)",0.019488,0.015163,0.007581
13,"(1, 4)","[0.592, 0.616, 0.616, 0.5725806451612904, 0.63...",0.606735,"(Age, Fare)",0.028608,0.022258,0.011129
3,"(3,)","[0.568, 0.6, 0.616, 0.6048387096774194, 0.6451...",0.6068,"(Parch,)",0.032064,0.024947,0.012473
31,"(1, 2, 3)","[0.648, 0.576, 0.656, 0.5887096774193549, 0.62...",0.617935,"(Age, SibSp, Parch)",0.040542,0.031543,0.015771
2,"(2,)","[0.624, 0.6, 0.656, 0.5645161290322581, 0.7177...",0.632452,"(SibSp,)",0.066955,0.052094,0.026047
23,"(0, 1, 4)","[0.656, 0.632, 0.648, 0.6129032258064516, 0.65...",0.640426,"(Pclass, Age, Fare)",0.020663,0.016076,0.008038
42,"(0, 1, 2, 4)","[0.656, 0.648, 0.64, 0.6129032258064516, 0.653...",0.642026,"(Pclass, Age, SibSp, Fare)",0.019983,0.015548,0.007774


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0, criterion='gini')
rfc.fit(x_train, y_train)
for feature in zip(x_train.columns, rfc.feature_importances_):
    print(feature)

('Pclass', np.float64(0.08580548090109545))
('Age', np.float64(0.26433757901109173))
('SibSp', np.float64(0.05355992352286061))
('Parch', np.float64(0.03699908167792097))
('Fare', np.float64(0.28790097031129963))
('Sex_male', np.float64(0.2713969645757317))


In [ ]:
from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(estimator=rfc, threshold=0.2)
sfm = sfm.fit(x_train, y_train)
feature_names=x_train.columns
print('The most important features based on random forest classifier:')
for feature_list_index in sfm.get_support(indices=True):
    print('- ' + feature_names[feature_list_index])

The most important features based on random forest classifier:
- Age
- Fare
- Sex_male


In [ ]:
from sklearn.metrics import accuracy_score
rfc_full = RandomForestClassifier(random_state=0, criterion='gini')
rfc_full.fit(x_train, y_train)
pred_full = rfc_full.predict(x_test)
print('The accuracy of classifier with full features: {:.2f}'.format(accuracy_score(y_test, pred_full)))

The accuracy of classifier with full features: 0.82


In [ ]:
x_important_train = sfm.transform(x_train)
x_important_test = sfm.transform(x_test)

In [ ]:
print(x_important_train.shape)
print(x_important_test.shape)

(623, 3)
(268, 3)


In [ ]:
rfc_lim = RandomForestClassifier(random_state=0, criterion='gini')
rfc_lim.fit(x_important_train, y_train)
pred_lim = rfc_lim.predict(x_important_test)
print('The accuracy of classifier with limited features: {:.2f}'.format(accuracy_score(y_test, pred_lim)))

The accuracy of classifier with limited features: 0.80


In [ ]:
df2=pd.read_csv('/content/house_price_train.csv')
df2.shape

(1460, 81)

In [ ]:
df2.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [ ]:
cols_to_drop = [
    'Id','Alley','MiscFeature','PoolQC','Fence','FireplaceQu',
    'Utilities','MiscVal','Condition2','GarageYrBlt',
    'TotalBsmtSF','GarageArea','MoSold'
]

In [ ]:
df2=df2.drop(cols_to_drop,axis=1)

In [ ]:
[col for col in df.columns if df[col].isnull().sum()>0]

[]

In [ ]:
X = df2.drop('SalePrice',axis=1)
y = df2['SalePrice']

In [ ]:
X = pd.get_dummies(X,drop_first=True)

In [ ]:
X.shape

(1460, 218)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
grouped_feature_ls = []
correlated_groups = []
def correlation (dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname=corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr
corr_features=correlation (x_test, 0.9)
correlated_features = set(corr_features)
print(len(correlated_features))

10


In [ ]:
x_train.drop(labels=correlated_features, axis=1, inplace=True)
x_test.drop(labels=correlated_features, axis=1, inplace=True)

In [ ]:
x_train.shape

(1022, 208)

In [ ]:
constant_features=[features for features in x_train.columns if x_train[features].std()==0]
len(constant_features)

2

In [ ]:
x_train.drop(labels=constant_features,axis=1,inplace=True)
x_test.drop(labels=constant_features,axis=1,inplace=True)

In [ ]:
quasi_constant_feat=[]

for feature in x_train.columns:
  predominant=(x_train[feature].value_counts()/float(len(x_train))).sort_values(ascending=False).values[0]
  if predominant>0.999:
    quasi_constant_feat.append(feature)

len(quasi_constant_feat)

17

In [ ]:
x_train.drop(labels=quasi_constant_feat,axis=1,inplace=True)
x_test.drop(labels=quasi_constant_feat,axis=1,inplace=True)

In [ ]:
dup=[]
for i in range (0,len(x_train.columns)):
    col1=x_train.columns[i]
    for col2 in x_train.columns[i+1:]:
        if x_train[col1].equals(x_train[col2]):
            dup.append(col2)
dup_features=set(dup)
print(len(dup_features))

0


In [ ]:
x_train.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'PavedDrive_Y', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw',
       'SaleType_New', 'SaleType_Oth', 'SaleType_WD', 'SaleCondition_Alloca',
       'SaleCondition_Family', 'SaleCondition_Normal'],
      dtype='object', length=189)

In [ ]:
import pandas as pd
from sklearn.feature_selection import mutual_info_regression, SelectKBest
mi = mutual_info_regression(x_train.fillna(0), y_train)
mi = pd.Series(mi, index=x_train.columns)
selector = SelectKBest(score_func=mutual_info_regression, k=10)
selector.fit(x_train.fillna(0), y_train)
selected_features = x_train.columns[selector.get_support()]
print("Top 10 Selected Features:\n", selected_features.tolist())
x_train_selected = selector.transform(x_train.fillna(0))
x_train_selected = pd.DataFrame(x_train_selected, columns=selected_features)

Top 10 Selected Features:
 ['MSSubClass', 'OverallQual', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath', 'GarageCars', 'ExterQual_TA']


In [ ]:
sfs3=SFS(RandomForestRegressor(n_jobs=4),k_features=5,forward=True,verbose=2,scoring='r2',cv=3)

In [ ]:
sfs3=sfs3.fit(np.array(x_train_selected.fillna(0)),y_train)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    6.4s finished

[2025-08-19 16:28:07] Features: 1/5 -- score: 0.6673562750390666[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    5.5s finished

[2025-08-19 16:28:13] Features: 2/5 -- score: 0.7231721070431775[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    5.7s finished

[2025-08-19 16:28:18] Features: 3/5 -- score: 0.7291772684488752[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    4.9s finished

[2025-08-19 16:28:23] Features: 4/5 -- score: 0.788394474166712[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    5.4s finished

[2025-08-19 16:28:29] Features: 5/5 -- score: 0.8125773460241416

In [ ]:
print('best accracy score',sfs3.k_score_)
print('best subset(indices)',sfs3.k_feature_idx_)
print('Best subset (corresponding names):', sfs3.k_feature_names_)

best accracy score 0.8125773460241416
best subset(indices) (1, 3, 5, 6, 8)
Best subset (corresponding names): ('1', '3', '5', '6', '8')


In [ ]:
sfs4=SFS(RandomForestRegressor(n_jobs=4),k_features=5,forward=False,verbose=2,scoring='r2',cv=3)
sfs4=sfs4.fit(np.array(x_train_selected.fillna(0)),y_train)
print('best accracy score',sfs4.k_score_)
print('best subset(indices)',sfs4.k_feature_idx_)
print('Best subset (corresponding names):', sfs4.k_feature_names_)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   10.7s finished

[2025-08-19 16:28:47] Features: 9/5 -- score: 0.8350610784949204[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    9.4s finished

[2025-08-19 16:28:56] Features: 8/5 -- score: 0.8283122059404621[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    8.4s finished

[2025-08-19 16:29:05] Features: 7/5 -- score: 0.8264787797459299[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    6.1s finished

[2025-08-19 16:29:11] Features: 6/5 -- score: 0.822148908195197

best accracy score 0.8145455597317088
best subset(indices) (0, 1, 3, 5, 6)
Best subset (corresponding names): ('0', '1', '3', '5', '6')


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    5.3s finished

[2025-08-19 16:29:16] Features: 5/5 -- score: 0.8145455597317088

In [ ]:
efs=EFS(RandomForestRegressor(n_jobs=4),min_features=1,max_features=4,scoring='r2',cv=5)
efs=efs.fit(x_train_selected,y_train)
print('best accuracy score',efs.best_score_)
print('best subset(indices)',efs.best_idx_)
print('Best subset (corresponding names):', efs.best_feature_names_)

Features: 385/385

best accuracy score 0.8048682273096786
best subset(indices) (1, 2, 5, 6)
Best subset (corresponding names): ('OverallQual', 'YearBuilt', '2ndFlrSF', 'GrLivArea')


In [ ]:
efs_results = pd.DataFrame.from_dict(efs.get_metric_dict()).T
efs_results.sort_values(by='avg_score', ascending=True, inplace=True)
efs_results

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
5,"(5,)","[0.3104163023544969, 0.05589299743493281, 0.16...",0.124113,"(2ndFlrSF,)",0.134779,0.104863,0.052431
4,"(4,)","[0.14710721842087082, 0.2261922528043201, 0.27...",0.14813,"(1stFlrSF,)",0.161422,0.125591,0.062796
3,"(3,)","[0.2311716040271503, 0.28557946642610843, 0.20...",0.221323,"(YearRemodAdd,)",0.063787,0.049629,0.024814
0,"(0,)","[0.28326858434012503, 0.2775444154498774, 0.17...",0.225765,"(MSSubClass,)",0.076959,0.059876,0.029938
12,"(0, 3)","[0.208440651600043, 0.3395366513699528, 0.2680...",0.2717,"(MSSubClass, YearRemodAdd)",0.053925,0.041955,0.020978
...,...,...,...,...,...,...,...
266,"(1, 2, 4, 6)","[0.6527072476996706, 0.8736761868236149, 0.769...",0.790598,"(OverallQual, YearBuilt, 1stFlrSF, GrLivArea)",0.098346,0.076516,0.038258
285,"(1, 3, 5, 6)","[0.6907238835089766, 0.853403268359594, 0.8031...",0.797573,"(OverallQual, YearRemodAdd, 2ndFlrSF, GrLivArea)",0.073806,0.057424,0.028712
280,"(1, 3, 4, 5)","[0.7248535125663562, 0.8378041949987876, 0.809...",0.799895,"(OverallQual, YearRemodAdd, 1stFlrSF, 2ndFlrSF)",0.050673,0.039425,0.019713
265,"(1, 2, 4, 5)","[0.7239966547695562, 0.84896264000877, 0.79863...",0.802881,"(OverallQual, YearBuilt, 1stFlrSF, 2ndFlrSF)",0.055106,0.042874,0.021437


In [143]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=0, criterion='squared_error')
rfr.fit(x_train, y_train)
for feature in zip(x_train.columns, rfr.feature_importances_):
    print(feature)

('MSSubClass', np.float64(0.0029478157624586586))
('LotFrontage', np.float64(0.009389239791431428))
('LotArea', np.float64(0.019686993886258962))
('OverallQual', np.float64(0.5565839953079916))
('OverallCond', np.float64(0.00559011256988828))
('YearBuilt', np.float64(0.019248658452805253))
('YearRemodAdd', np.float64(0.007255883796809439))
('MasVnrArea', np.float64(0.014368011606219547))
('BsmtFinSF1', np.float64(0.03427012049872494))
('BsmtFinSF2', np.float64(0.0010343020039719295))
('BsmtUnfSF', np.float64(0.008051941569982518))
('1stFlrSF', np.float64(0.04554019990772116))
('2ndFlrSF', np.float64(0.009062621789479156))
('LowQualFinSF', np.float64(8.291837497339758e-05))
('GrLivArea', np.float64(0.12933999600023888))
('BsmtFullBath', np.float64(0.0009172222415062764))
('BsmtHalfBath', np.float64(0.0008851025764347891))
('FullBath', np.float64(0.0030700678285506735))
('HalfBath', np.float64(0.0012071219627734515))
('BedroomAbvGr', np.float64(0.004089291610617657))
('KitchenAbvGr', np.

In [144]:
from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(estimator=rfr, threshold=0.2)
sfm = sfm.fit(x_train, y_train)
feature_names=x_train.columns
print('The most important features based on random forest classifier:')
for feature_list_index in sfm.get_support(indices=True):
    print('- ' + feature_names[feature_list_index])

The most important features based on random forest classifier:
- OverallQual


In [145]:
from sklearn.metrics import mean_squared_error, r2_score
rfr_full = RandomForestRegressor(random_state=0, criterion='squared_error')
rfr_full.fit(x_train, y_train)
pred_full = rfr_full.predict(x_test)
print("R² Score:", r2_score(y_test, pred_full))
print("MSE:", mean_squared_error(y_test, pred_full))

R² Score: 0.8664108554774486
MSE: 906935558.7691807


In [146]:
x_important_train = sfm.transform(x_train)
x_important_test = sfm.transform(x_test)

In [147]:
print(x_important_train.shape)
print(x_important_test.shape)

(1022, 1)
(438, 1)


In [148]:
rfr_lim = RandomForestRegressor(random_state=0, criterion='squared_error')
rfr_lim.fit(x_important_train, y_train)
pred_lim = rfc_lim.predict(x_important_test)
print("R² Score (limited features):", r2_score(y_test, pred_lim))
print("MSE (limited features):", mean_squared_error(y_test, pred_lim))

R² Score (limited features): 0.6914271092644155
MSE (limited features): 2094898714.1171165
